In [1]:
from neo4j import GraphDatabase

uri = 'neo4j://localhost:7687'
user = 'neo4j'
password = 'password'

driver = GraphDatabase.driver(uri, auth=(user, password))

1. Найти самый старый фильм и самый новый фильм - вывести их названия по 1 шт (2 запроса)

In [ ]:
def find_oldest_movie(tx):
    result = tx.run("""
        MATCH (movie:Movie)
        WITH MIN(movie.released) as oldest_year
        MATCH (old_movie:Movie {released: oldest_year})
        RETURN collect (old_movie.title) as movie_title
    """)
    return result.single()["movie_title"]

#использовала в запросе collect на случай, если в том же году было снято несколько фильмов

with driver.session() as session:
    movie_title = session.execute_read(find_oldest_movie)
    print(movie_title)

["One Flew Over the Cuckoo's Nest"]


In [3]:
def find_newest_movie(tx):
    result = tx.run("""
        MATCH (movie:Movie)
        WITH MAX(movie.released) as newest_year
        MATCH (new_movie:Movie {released: newest_year})
        RETURN collect (new_movie.title) as movie_title
    """)
    return result.single()["movie_title"]

with driver.session() as session:
    movie_title = session.execute_read(find_newest_movie)
    print(movie_title)

['Cloud Atlas']


2. Получить среднее количество актёров на фильм

In [4]:
def find_avg_actor_count(tx):
    result = tx.run("""
        MATCH (movie:Movie)<-[:ACTED_IN]-(actor:Person)
        WITH movie, COUNT(actor) as actor_count
        RETURN AVG(actor_count) as avg_actor_count
    """)
    return result.single()["avg_actor_count"]

with driver.session() as session:
    avg_actor_count = session.execute_read(find_avg_actor_count)
    print(avg_actor_count)

4.526315789473684


3. Группировка фильмов по годам и подсчёт количества фильмов в каждом году

In [5]:
def year_movie_count(tx):
    result = tx.run("""
        MATCH (movie:Movie)
        WITH movie.released as year, COUNT(movie.title) as movie_count
        RETURN year, movie_count ORDER BY year
    """)
    return [record for record in result]

with driver.session() as session:
    movie_count = session.execute_read(year_movie_count)
    for m in movie_count:
        print(f'{m["year"]} - {m["movie_count"]}')

1975 - 1
1986 - 2
1990 - 1
1992 - 4
1993 - 1
1995 - 2
1996 - 3
1997 - 2
1998 - 3
1999 - 4
2000 - 3
2003 - 3
2004 - 1
2006 - 3
2007 - 1
2008 - 2
2009 - 1
2012 - 1


4*. Найти актёра с наибольшим промежутком между первым и последним фильмом

In [6]:
def actor_max_movie_gap(tx):
    result = tx.run("""
        MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie)
        WITH actor, MIN(movie.released) as oldest_year, MAX(movie.released) as newest_year
        WITH actor, newest_year-oldest_year as movie_gap
        RETURN actor.name as actor_name, movie_gap ORDER BY movie_gap DESC LIMIT 1
    """)
    return [record for record in result]

with driver.session() as session:
    movie_gap = session.execute_read(actor_max_movie_gap)
    for m in movie_gap:
        print(f'{m["actor_name"]} - {m["movie_gap"]}')

Jack Nicholson - 28
